In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

c:\Users\Balaji\Desktop\ANN Classifier\venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
df=pd.read_csv("Churn_Modelling.csv")

In [3]:
df=df.drop(["RowNumber","CustomerId","Surname"],axis=1)
label_encoder_gender=LabelEncoder()
df["Gender"]=label_encoder_gender.fit_transform(df[["Gender"]])
one_hot_encoder=OneHotEncoder()
geo_encoder=one_hot_encoder.fit_transform(df[["Geography"]])
geo_encoder.toarray()
geo_encoded_pd=pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encoder.get_feature_names_out(["Geography"]))
df=pd.concat([df.drop("Geography",axis=1),geo_encoded_pd],axis=1)


x=df.drop("Exited",axis=1)
y=df["Exited"]

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)
with open("one_hot_encoder.pkl","wb") as file:
    pickle.dump(one_hot_encoder,file)
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)



c:\Users\Balaji\Desktop\ANN Classifier\venv\Lib\site-packages\sklearn\preprocessing\_label.py:120: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
## define a fucnction to create the model and try different parameters(KerasClassifier)
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation="relu",input_shape=(x_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation="relu"))
    
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    return model

In [5]:
## create a keras classifier
model=KerasClassifier(neurons=32,layers=1,build_fn=create_model,verbose=0)

In [6]:
## define the grid search params
param_grid={
    "neurons":[16,32,64,128],
    "layers":[1,2,3],
    "epochs":[50,100]
}

In [ ]:
# perform grid search
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)
grid_result=grid.fit(x_train,y_train)

print("Best: %f using %s"%(grid_result.best_score_,grid_result.best_params_))